# Verarbeitung GHX Kataloge (WorkFlow)

In [61]:
## 1. Zugriff auf FTP
#- In Klärung BW

## 2. Liste alle zips in FTP

## 3. Liste alls zips in GHX-Kataloge-Arbeitsverzeichnis

## 4. Bestimme relevante (neue, noch nicht in GHX-Kataloge-Arbeitsverzeichnis aufgeführte) zips in FTP

## 5. Kopiere relevante zips in GHX-Kataloge-Arbeitsverzeichnis

## 6. Entpacke relevante zips "chronologisch" nach Datum im Dateinamen 
#- (Jede zip durchläuft nacheinander die Schritte 7 -9)

## 7. Lese .csv Dateien in Datenbank ein

## 8. Schreibe verarbeitete .zip in tbl_zips in die DB

## 9. Lösche verarbeitete Ordner mit csv

In [62]:
import os
import os.path
import glob
import shutil
import pandas as pd
import numpy as np
import datetime as dt
import zipfile
import gc
import time
from sqlalchemy import create_engine
import sqlalchemy
import urllib
import import_ipynb
import ipynb
import pyodbc
import schedule
from smb.SMBConnection import SMBConnection


# Import eigene Klasse
#import py_etl_sql_funktionen
#from ipynb.fs.full.py_etl_sql_funktionen import sql_ausfuehrung



#### 2. Liste alle zips in FTP

- Lese Zip Dateien ab initialer zip aufsteigend
- Versehe mit Zeitstempel und lade zukünftige csv Dateien nur ein, wenn noch nicht in Zip-Liste enthalten

## 1. Zugriff auf FTP
- In Klärung BW

In [63]:
current_path = "".join([os.getcwd(), "\\"]) 
etl_path = "Z:\\1_AGKAMED_Arbeit\\0_GIT_REPOS\\1_ETL\\"
ghx_cataloge_path = "".join([etl_path,"GHX_Kataloge\\"])
# ghx_cataloge_path = "".join([current_path,"GHX_Kataloge\\"])
# ftp_path = "".join([current_path,"GHX_FTP\\"]) 
ini_zip = 'CatUpd_AGKAMED_20200717030056.zip'

In [64]:
# type(ftp_path)

str

## 2. Liste alle zips in FTP

## 3. Liste alls zips in GHX-Kataloge-Arbeitsverzeichnis

In [65]:
# Konstanten für SMB Zugriff
# userID = 'ftp-serv-ip3'
# password = 'bnT0RAOo1DN5NkokQbCs'
# client_machine_name = 'AGKVMPC6' # local PC Name
# server_name = 'agkftp01'
# server_ip = '192.168.209.42'
shared_folder_name = 'Ghx_kataloge$'

In [66]:
def _smb_connector():
    """
    Stellt Verbindung zu ftp-Ordner (GHX-Kataloge) via SMB her.
    INPUT:
        --
    OUTPUT:
        conn: Verbindung
    """
    userID = 'ftp-serv-ip3'
    password = 'bnT0RAOo1DN5NkokQbCs'
    client_machine_name = 'AGKVMPC6' # local PC Name
    server_name = 'agkftp01'
    server_ip = '192.168.209.42'
    
    conn = SMBConnection(userID, password, client_machine_name, server_name, use_ntlm_v2=True,
                         is_direct_tcp=True)

    conn.connect(server_ip, 445) 

    if conn:
        # print("\nSMB-Verbindung zu {} erfolgreich: ".format(conn))
        return conn

    else:
        print("SMB-Verbindung fehlerhaft.")

    
    conn.close()
    

### Funktionen zur Datei-Listung

In [67]:
def _dateien_in_ftp_smb():
    """
    Auflistung von .zip-Dateien in SMB Ordner 
    INPUT: 
        SMB-Verbindung zum remote-ftp-Kataloge-Ordner
    """
    initial_zip = ini_zip
    conn = _smb_connector()
    lst_smb = conn.listPath(shared_folder_name,'/',pattern='*.zip')
    lst_smb_zips = [share.filename for share in lst_smb if share.filename[15:25] >= initial_zip[15:25]]
    return lst_smb_zips # Liste aller im Ordner befindlichen .zips ab Initial-Zip

In [68]:
def _dateien_in_ordner(pfad,suffix='.zip'):
    """
    Auflistung von Dateien jeglichen Typs in beliebigem Ordner
    
    INPUT: 
        - Ordner 
        - Dateityp (Vorbelegt mit zip) "Alle Typen möglich"
    OUTPUT:
        - Liste der Dateien des gewählten Typs in dem angegebenen Ordner
    """
    
    fil_lst = [fil for fil in os.listdir(pfad) if fil.endswith(suffix)]
    return fil_lst
    

In [69]:
def _start_zip_handler():
    """
    Prüfung, ob die gefundenen .zips Dateien in beiden Ordnern übereinstimmen.
    Bei Übereinstimmung wird Programm beendet. 
    Sonst startet zip-Handler.
    INPUT: 
        quellPfad,zielPfad = Vorbelegt mit SMB Ordner Inhalt und Arbeitsordner
    OUTPUT:
        boolscher Wert 
    """
    vgl_lsts = set(_dateien_in_ftp_smb()) == set(_dateien_in_ordner(ghx_cataloge_path))

    return vgl_lsts

In [70]:
_start_zip_handler()

## 4. Finde relevante (neue, noch nicht in GHX-Kataloge-Arbeitsverzeichnis aufgeführte) zips in FTP


In [71]:
def _neustes_datum_ghx_zip(pfad):
    """
    Lese aktuellstes Datum der bereits vorliegenden zips in GHX Kataloge Arbeitsordner.
    Falls keine zip-Datei im GHX-Arbeitsordner wird 
    auf das Datum der Initialen zip zurückgegriffen.
    
    INPUT:
        Liste mit ZIP-Dateien "GHX-Kataloge" 
    OUTPUT:
        Aktuellstes Datum
        
    """
    initial_zip = ini_zip
    if not _dateien_in_ordner(pfad):
        return initial_zip[15:25]
    else: 
        df_dat = pd.DataFrame(_dateien_in_ordner(pfad)
                              ,dtype=str
                              ,columns=['Zip_Name_ID'])
        df_dat['Zip_Datum'] = df_dat['Zip_Name_ID'].str.slice(start=15, stop=25)
        return df_dat['Zip_Datum'].max()


In [72]:
def _erstelle_inload_liste():
    """
    Liste der relevanten ZIP Dateien erstellen, in Abhängigkeit
    der bereits vorliegenden ZIPs im GHX-Kataloge-Arbeitsordner.
    Liegt keine zip im Arbeitsordner werden alle im ftp befindlichen Dateien
    ab dem intial-zip auf die Inload-Liste gesetzt und eingeladen.  
    
    INPUT:
        quellPfad und zielPfad: ftp oder ghx-Arbeitsordner 
    OUTPUT:
        Liste relevanter ZIPs für Inload (Typ: Liste)
    """
    
    relev_zips_lst = []

    if not _dateien_in_ordner(ghx_cataloge_path): # Wenn GHX-ArbeitsOrdner leer ist
        zip_dateien = _dateien_in_ftp_smb()
        print("GHX-Arbeitsordner ist leer.\nEs werden alle .zip-Dateien aus FTP\n"
              "ab dem Initialdatum geladen.")
        for datei in zip_dateien:
            if datei[15:25] >= _neustes_datum_ghx_zip(ghx_cataloge_path):
                try:
                    relev_zips_lst.append({'Zip_Name_ID' : datei, 
                                   'Zip_Datum'   : datei[15:25],
                                   'Zeitstempel' : dt.datetime.now()
                                   })
                except Exception as e:
                    print(e)
                    continue
        return relev_zips_lst
    else: 
        zip_dateien = _dateien_in_ftp_smb()
        for datei in zip_dateien:
            if datei[15:25] > _neustes_datum_ghx_zip(ghx_cataloge_path):
                try:
                    relev_zips_lst.append({'Zip_Name_ID' : datei, 
                                   'Zip_Datum'   : datei[15:25],
                                   'Zeitstempel' : dt.datetime.now()
                                   })
                except Exception as e:
                    print(e)
                    continue
        return relev_zips_lst 

In [73]:
df = pd.DataFrame(_erstelle_inload_liste())
df

## 5. Kopiere relevante zips in GHX-Kataloge-Arbeitsverzeichnis

In [74]:
# def _kopiere_relev_zips(zipDF,zipNamenFeld):
#     """
#     Kopiert alle relevanten .zip-Dateien auf ftp-Ordner in GHX-Kataloge-Arbeitsordner.
    
#     INPUT:
#         quellPfad:   ftp, 
#         zielPfad:    ghx-Arbeitsordner (Vorbelegt),
#         zipDF:       DF der relevanten zips (Erweiterter Scope), 
#         zipNamenFeld:Spaltenname der Zip-Dateien
#     OUTPUT: 
#         --
#     """
    
#     df_relev_zips = pd.DataFrame(zipDF,dtype=str) # ok
#     relev_zips = df_relev_zips[zipNamenFeld].tolist() # ok

#     zips_ftp = _dateien_in_ftp_smb()
    
#     # #############
#     # ANPASSEN! String "quellPfad" durch SMB-Verbindung
#     # #############
    
#     [shutil.copy(quellPfad + fil,ghx_cataloge_path + fil) for fil in zips_ftp if fil in relev_zips] 
    


In [75]:
def _kopiere_relev_zips_from_smb():
        """ Download files from the remote share. """
        conn = _smb_connector()
        df_smb_files = pd.DataFrame(_erstelle_inload_liste(),dtype=str) # Liste relevante Zips
        server_name = 'agkftp01'
        shared_folder_name = 'Ghx_kataloge$'
        lst_smb_files = df_smb_files['Zip_Name_ID'].to_list()

        for file in lst_smb_files:
            try: 
                with open(file, 'wb') as file_obj:
                    conn.retrieveFile(
                                      service_name=shared_folder_name,
                                      path=file,
                                      file_obj=file_obj)
                    
                shutil.move(current_path + file, ghx_cataloge_path + file) # Schiebt .zip-Dateien von Programm-Verzeichnis ins Arbeitsverzeichnis.
            except Exception as OpF:
                print("\n" + str(OpF) + "\n")
        
#         [shutil.move(current_path + fil,ghx_cataloge_path + fil) for fil in lst_smb_files] 
        print("\n\nZIPs wurden erfolgreich in den Arbeitsordner kopiert.\n")

## 6. Entpacke relevante zips "chronologisch" nach Datum im Dateinamen 
- (Jede zip durchläuft nacheinander die Schritte 7 -9)

In [76]:
def _extract_folder_from_zips(zipDF,zipNamenFeld):
    """
    Entpacke ZIP's
    
    INPUT: 
        Pfad mit ZIP's
    OUTPUT:
        --
    """
    
    df_relev_zips = pd.DataFrame(zipDF,dtype=str)
    relev_zips = df_relev_zips[zipNamenFeld].tolist()
    
    for zip_file in relev_zips:
        try:
            with zipfile.ZipFile(ghx_cataloge_path + zip_file,'r') as myZip:
                myZip.extractall(path=ghx_cataloge_path)
                _fill_csv_in_ms_sql()
                _del_fold_csv(ghx_cataloge_path)
                gc.collect(generation=2)
                print("Neu importierte zip: " + zip_file)
        except Exception as e:
            print(e)
            continue 

## 7.1 Lese .csv Dateien in Datenbank ein


In [77]:
def _verbinde_ms_sql():
    server_login = urllib.parse.quote_plus(r"DRIVER={SQL Server Native Client 11.0};"
                                r"SERVER=192.168.16.124;"
                                r"DATABASE=Vorlauf_DB;"
                                r"Trusted_Connection=yes;")   
    server_verbindung = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(server_login))
    return server_verbindung

#                                r"SERVER=192.168.16.124;"
#                                r"DATABASE=Vorlauf_DB;"
#                                r"SERVER=192.168.16.123;"
#                                r"DATABASE=Vorlauf_DB;"

In [78]:
# type(_verbinde_ms_sql()) # sqlalchemy.engine.base.Engine
# type(_verbinde_zu_server_und_db()) # pyodbc.Connection

In [79]:
def _fill_csv_in_ms_sql():
    """
    Fülle csv in etl Datenbank
    INPUT:
        csv:    GHX Kataloge        
    OUTPUT:
        --
    """
    chunk_size = 10000
    i = 1
    j = 1
    start_time = time.time() 

    server_verbindung = _verbinde_ms_sql()
    alle_csv = glob.glob("**/*.csv",recursive=True)
    for csv_file in alle_csv:
        try:
            for df_csv in pd.read_csv(csv_file,
                                sep='\t',
                                # dtype=str,
                                dtype='unicode',
                                lineterminator='\r',
                                iterator=True,
                                chunksize=chunk_size,
                                encoding='latin-1'
                                ):
                df_csv = df_csv.rename(columns = {c: c.replace(' ','') for c in df_csv.columns})
                df_csv.index += j
                # Spalten ergänzen / bereinigen
                df_csv['TPSHORTNAME'] = df_csv['TPSHORTNAME'].str.replace(r'\W','')
                df_csv['_computed_normalized_suppliername_'] = df_csv['SUPPLIERNAME'].str.replace(r'\W','')
                df_csv['_katalog_inload_'] = dt.datetime.now()
                df_csv['_computed_normalized_art_nr_'] = df_csv['SUPPLIERPARTNUM'].str.replace(r'\W','')
                df_csv['_artikel_key_inload_'] = df_csv['_computed_normalized_suppliername_'] + "°" + df_csv['_computed_normalized_art_nr_'] + "°" + df_csv['UOM'] + "°" + df_csv['NOU']
                df_csv['_csv_file_'] = csv_file.split("\\",1)[1]
                df_csv['_prio_flag_'] = ""
                df_csv['_hibc_'] = ""
                df_csv['_gtin_'] = ""
                df_csv['_ean_kommentar_'] = ""
                # Übertrag aus sql_empty_cols
                df_csv['_LieferantenNameGlobal_'] = ""
                df_csv['_artikel_key_'] = ""
                df_csv['_AGKA_Kreditoren_ID_'] = ""
                df_csv['_artikel_key_class_'] = ""
                df_csv['_agka_class_id_'] = ""
                df_csv['_UStID_Vorschlag_'] = ""
                
                df_csv['_gtin_pruefziffer_'] = ""
                df_csv['_gtin_pruefung_kommentar_'] = ""
                
                
                df_csv['_AGKclassCode-LEVEL1_'] = ""
                df_csv['_AGKclassCode-LEVEL2_'] = ""
                df_csv['_AGKclassCode-LEVEL3_'] = ""
                df_csv['_AGKclassCode-LEVEL4_'] = ""
                
#                df_csv['tbl_index'] = ""
                
                #df_inload_verw = df_csv[['TPSHORTNAME','_artikel_key_inload_','_csv_file_','_katalog_inload_']]
                #df_csv = df_csv.dropna(subset = ['MFRNAME','SUPPLIERNAME','SUPPLIERPARTNUM','NOU','UOM'],inplace=True) #  Zeilen mit Nullwerten ignorieren
                df_csv.to_sql("tbl_ghx_kataloge_roh",con=server_verbindung,if_exists='append',index=False)
                #df_inload_verw.to_sql("tbl_ghx_inload_verw",con=server_verbindung,if_exists='append',index=False)
                j = df_csv.index[-1]+1
                end_time = time.time()
                print("Dauer je Chunk (Zeilen-Paket): ", end_time - start_time)


                
        except Exception as e:
            print(csv_file)
            print(e)
            continue            

## 7.2 Lese xlsx Dateien in Datenbank ein

#### xlsx Tabellen

In [80]:
xlsx_tab_names = ["tbl_covin_validierung",
                  "tbl_lieferanten",
                  "tbl_lieferanten_master",
                  "tbl_artikel_agkaClass_jd",
                  "tbl_agkaClass_jd",
                  "tbl_warenkorb",
                  "tbl_abdata"]

#### Zu verändernde Spalten je xlsx Tabelle 

In [81]:
dictSpaltenZurAnpassung = { "tbl_warenkorb":(   'LieferantenNrAGKA',
                                                'LieferantenNameGlobal',
                                                'LieferantenArtikelNr'),
                            "tbl_lieferanten_master":( 'TPSHORTNAME',
                                                'LieferantenNr',
                                                'MiiLieferantGlobal',
                                                'Finale_UStID'),
                            "tbl_artikel_agkaClass_jd":('Lieferantennummer_vom_Lieferanten',
                                                        'Artikelnummer_vom_Lieferanten'),
                            "tbl_lieferanten_ueberl":('LieferantenNr',
                                                      'Nachfolger'),
                            "tbl_lieferanten_kataloge":('TPSHORTNAME',
                                                        'LieferantenNr')}
# tbl_warenkorb add 'LieferantenNrAGKA',

In [82]:
def _import_xlsx_conv_to_df(tabelle):
    """
    Einlesen xlsx-Stammdatentabellen in Vorlauf_DB Datenbank
    INPUT:
        Tabellenname (.xlsx)
    OUTPUT:
        xlsx-tab als Datenframe
    """
    df_xlsx = pd.read_excel(current_path + tabelle + ".xlsx",sheet_name="_Importdaten",dtype=str)
    df_xlsx = df_xlsx.assign(tbl_index=df_xlsx.index + 1) # Setze nachträglich Index-Spalte des Typs int
    return df_xlsx

In [83]:
def _import_xlsx_inkl_sonderZ_Ber(tabelle):
    '''
    Automatische Bereinigung auf Sonderzeichen in den genannten Spalten (Loop) in dictSpaltenZurAnpassung

    INPUT:
        Tabellenname (.xlsx)
    OUTPUT:
        Datenframe inkl. bereinigter Spalten
    
    '''
    df_xlsx = _import_xlsx_conv_to_df(tabelle)
    
    df_xlsx = df_xlsx.rename(columns = {c: c.replace(' ','') for c in df_xlsx.columns})
    
    
    if tabelle in dictSpaltenZurAnpassung:
        for spalte in range(0,len(dictSpaltenZurAnpassung[tabelle])):
            spl = dictSpaltenZurAnpassung[tabelle][spalte]
            df_xlsx[spl] = df_xlsx[spl].str.replace(r'\W','')
            print(spl)
        
        return df_xlsx
            
    else:
        print("Tabelle {} enthält keine zu \nbereinigenden Spalten.".format(tabelle))
        #continue

    return df_xlsx


In [84]:
def _fill_xlsx_in_ms_sql(tabelle):
    """
    Einlesen xlsx in Vorlauf_DB Datenbank tabs

    INPUT:
        Tabellenname:
        xlsx:   Covin Validierungsregeln 
                Lieferanten
                Artikel Zuordnung agka class
                agka class
                Überleitung agka zu eClass
                Individueller Warenkorb
    OUTPUT:
        --
    """
    chunk_size = 10000
    start_time = time.time() 
    server_verbindung = _verbinde_ms_sql()
    

    
    try:
        df_xlsx = _import_xlsx_inkl_sonderZ_Ber(tabelle) # Übergabe des importierten, bereinigten DF
        if tabelle == "tbl_lieferanten_master":
            df_xlsx_lief = df_xlsx[['LieferantenNr'
                                    ,'AGKALieferanten_Lieferant'
                                    ,'MiiLieferantGlobal'
                                    ,'AGKAMED_Langname'
                                    ,'Finale_UStID'
                                    ,'tbl_index']]
#            df_xlsx_lief = df_xlsx_lief.assign(tbl_index=df_xlsx.index + 1) # Setze nachträglich Index-Spalte des Typs int
            df_xlsx_lief = df_xlsx_lief.drop_duplicates(subset=None, keep='first', inplace=False)
            df_xlsx_lief_kat = df_xlsx[['TPSHORTNAME'
                                        ,'LieferantenNr'
                                        ,'tbl_index']]
            df_xlsx_lief.to_sql('tbl_lieferanten',con=server_verbindung,if_exists='replace',index=False)
            df_xlsx_lief_kat.to_sql('tbl_lieferanten_kataloge',con=server_verbindung,if_exists='replace',index=False)

        elif tabelle == 'tbl_artikel_agkaClass_jd':
            df_xlsx['_artikel_key_class_'] = ''
            df_xlsx.to_sql('tbl_artikel_agkaClass_jd',con=server_verbindung,if_exists='replace',index=False)

        else:
            df_xlsx.to_sql(tabelle,con=server_verbindung,if_exists='replace',index=False)
    except Exception as e:
        print(e)

## 9. Lösche verarbeitete Ordner mit csv

In [85]:
def _find_csv_fold_for_deletion(pfad):
    """
    Listet entpackte Ordner mit csv-Dateien.
    
    INPUT:
        pfad: Pfad aus welchem Ordner gelöscht werden sollen. Hier i. d. R. ghx_cataloge_path.
    OUTPUT:
        lst_for_del: Liste der Ordner in Arbeitsordner
    """
    
    pfad = next(os.walk(pfad))[0]
    ordner_in_ftp =  next(os.walk(pfad))[1]
    lst_for_del = [pfad + ordner for ordner in ordner_in_ftp]
    return lst_for_del

In [86]:
def _del_fold_csv(pfad):
    """
    Löscht durch _find_csv_fold_for_deletion erkannte Ordner aus Arbeitsordner.
    
    INPUT:
        lst_for_del: Ausführung der Funktion zur Bestimmung der zu löschenden Ordner.
        pfad: Pfad aus welchem Ordner gelöscht werden sollen. Hier i. d. R. ghx_cataloge_path.
    OUTPUT:
        --
    """
    lst_for_del = _find_csv_fold_for_deletion(pfad)
    if not lst_for_del:
        print("Keine csv-Ordner in Arbeitsordner vorhanden."
             "\nFunktion wird beendet.")
        return
    else:
        try:
            [shutil.rmtree(fold) for fold in lst_for_del]
        except Exception as e:
            print(e)

# SQL Skripte Umgebung

In [87]:
def _verbinde_zu_server_und_db():
    with pyodbc.connect(r"DRIVER={SQL Server Native Client 11.0};"
                                "SERVER=192.168.16.124;"
                                "DATABASE=Vorlauf_DB;"
                                "Trusted_Connection=yes;") as verb_db:

        return verb_db
#                                "SERVER=192.168.16.124;"
#                                "DATABASE=Vorlauf_DB;"

In [88]:
def _sql_executer(sql_string):
    db_verb = _verbinde_zu_server_und_db()
#     db_verb = _verbinde_ms_sql()
    cur = db_verb.cursor()
    cur.execute(sql_string)
    db_verb.commit()

#     Vorlage: Alternativer sql_executer über sqlalchemy 
#     with _verbinde_ms_sql().connect() as connection:
#         result = connection.execute(sql_delete_empty_rows)


In [89]:
def _export_warenkorb():
    """
    Importiert einen Warenkorb in die Datenbank.
    
    INPUT:
       Funktion _fill_xlsx_in_ms_sql für Tabelle Warenkorb
    OUTPUT:
        --
    """
    start = time.process_time()
    
    _fill_xlsx_in_ms_sql('tbl_warenkorb') # Import Warenkorb in DB 
    _sql_executer(sql_export_warenkorb) # Ausführung Query Mapping Warenkorb mit tbl_kataloge_roh
    db_verb = _verbinde_zu_server_und_db() 
#    df_exp_xlsx = pd.read_sql(sql_export_warenkorb, db_verb) # Funktioniert bei kleinen Tabellen 

    df_exp_xlsx = pd.DataFrame()
    for df_teil in pd.read_sql(sql_export_warenkorb,db_verb, chunksize = 3):
        df_exp_xlsx = df_exp_xlsx.append(df_teil,ignore_index=True)
    # Nachträgliche Anpassungen des Warenkorbs 
    df_exp_xlsx.drop_duplicates(subset=['Index'
                                        ,'MFRNAME'
                                        ,'MFRPARTNUM'
                                        ,'SUPPLIERNAME'
                                        ,'SUPPLIERPARTNUM'
                                        ,'BaseUOM'
                                        ,'NOU'
                                        ,'UOM' 
                                        ,'_prio_flag_' 
                                        ,'_artikel_key_'], inplace=True) # keep='false' 
#     df_exp_xlsx.query('_prio_flag_ != "0"',inplace = True)
    df_exp_xlsx.to_excel("KATALOGMAPPING_eCl@ss.xlsx") # Export Warenkorb in xlsx
    print('\nWarenkorb Mapping abgeschlossen.\ndie KATALOGMAPPING_eCl@ss.xlsx Datei liegt bereit.')
    print(time.process_time() - start)
    db_verb.commit()

In [ ]:
def _export_warenkorb_gtin():
    """
    Importiert einen Warenkorb in die Datenbank.
    
    INPUT:
       Funktion _fill_xlsx_in_ms_sql für Tabelle Warenkorb
    OUTPUT:
        --
    """
    start = time.process_time()
    
    _fill_xlsx_in_ms_sql('tbl_warenkorb') # Import Warenkorb in DB 
    _sql_executer(sql_export_warenkorb_gtin) # Ausführung Query Mapping Warenkorb mit tbl_kataloge_roh
    db_verb = _verbinde_zu_server_und_db() 
#    df_exp_xlsx = pd.read_sql(sql_export_warenkorb, db_verb) # Funktioniert bei kleinen Tabellen 

    df_exp_xlsx = pd.DataFrame()
    for df_teil in pd.read_sql(sql_export_warenkorb_gtin,db_verb, chunksize = 3):
        df_exp_xlsx = df_exp_xlsx.append(df_teil,ignore_index=True)
    # Nachträgliche Anpassungen des Warenkorbs 
    df_exp_xlsx.drop_duplicates(subset=['Index'
                                        ,'MFRNAME'
                                        ,'MFRPARTNUM'
                                        ,'SUPPLIERNAME'
                                        ,'SUPPLIERPARTNUM'
                                        ,'BaseUOM'
                                        ,'NOU'
                                        ,'UOM' 
                                        ,'_prio_flag_' 
                                        ,'_artikel_key_'], inplace=True) # keep='false' 
#     df_exp_xlsx.query('_prio_flag_ != "0"',inplace = True)
    df_exp_xlsx.to_excel("KATALOGMAPPING_eCl@ss.xlsx") # Export Warenkorb in xlsx
    print('\nWarenkorb Mapping abgeschlossen.\ndie KATALOGMAPPING_eCl@ss.xlsx Datei liegt bereit.')
    print(time.process_time() - start)
    db_verb.commit()

In [90]:
db_tabellen = ["[Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]"
               ,"[Vorlauf_DB].[dbo].[tbl_verw_inload_kataloge]"
               ,"[Vorlauf_DB].[dbo].tbl_verw_csv"
               ,"[Vorlauf_DB].[dbo].tbl_verw_zip"
               ,"[Vorlauf_DB].[dbo].tbl_verw_ean"
               ,"[Vorlauf_DB].[dbo].tbl_pzn_lauertaxe"
               ,"[Vorlauf_DB].[dbo].tbl_lieferanten"
               ,"[Vorlauf_DB].[dbo].tbl_artikel_agkaClass_jd"]
db_tabellen[7]

'[Vorlauf_DB].[dbo].tbl_artikel_agkaClass_jd'

### Warenkorb: Mapping GTIN Ebene 

In [91]:
sql_export_warenkorb_gtin = """
WITH cte_warenkorb_map AS (
SELECT [Index]
,[LieferantenNrAGKA]
,[LieferantenNameGlobal]
,[LieferantenArtikelNr]
,[LieferantenArtikelBeschreibung]
,[NOU]
,[UOM]
,CONCAT(LieferantenNameGlobal,'°',LieferantenArtikelNr,'°',[UOM],'°',[NOU]) AS ARTIKEL_KEY_WARENKORB
FROM [Vorlauf_DB].[dbo].[tbl_warenkorb]
),
ghx_kompl AS (
SELECT cte_warenkorb_map.[Index]
      ,cte_warenkorb_map.ARTIKEL_KEY_WARENKORB
      ,[TPSHORTNAME]
      ,[ACTIONCODE]
      ,[LOCALE]
      ,[MFRNAME]
      ,[MFRPARTNUM]
      ,[SUPPLIERNAME]
      ,[SUPPLIERPARTNUM]
      ,[BaseUOM]
      ,tbl_roh.[NOU]
      ,tbl_roh.[UOM]
      ,[SHORTDESC]
      ,[LISTPRICE]
      ,[TAXWARECOMMODITYCODE]
      ,[LONGDESC]
      ,[SEARCHABLEKEYWORD]
      ,[EUROPMEDICALDEVICEDIRECTIVECLASS]
      ,[PACKAGINGINFORMATION]
      ,[EAN]
      ,[PZN]
      ,[LOTSIZE]
      ,[MINORDER]
      ,[MAXORDER]
      ,[LEADTIME]
      ,[AVAILABILITY]
      ,[NETWEIGHT]
      ,[GROSSWEIGHT]
      ,[WEIGHTUOM]
      ,[PRODUCTWIDTH]
      ,[PRODUCTHEIGHT]
      ,[PRODUCTDEPTH]
      ,[PACKAGEDWIDTH]
      ,[PACKAGEDHEIGHT]
      ,[PACKAGEDDEPTH]
      ,[DIMENSIONUOM]
      ,[CATLEVEL1]
      ,[CATLEVEL2]
      ,[CATLEVEL3]
      ,[CATLEVEL4]
      ,[CATLEVEL5]
      ,[CATNAME]
      ,[CATVERSION]
      ,[CATCODE]
      ,[OTHERPARTNUMNAME1]
      ,[OTHERPARTNUM1]
      ,[OTHERPARTNUMNAME2]
      ,[OTHERPARTNUM2]
      ,[DIVISION]
      ,[HAZMATFLAG]
      ,[RECYCLEDMATERIALS]
      ,[SAFETYFLAG]
      ,[RESTRICTFLAG]
      ,[RESTRICTTYPES]
      ,[PURPOSE1]
      ,[LOCATION1]
      ,[TYPE1]
      ,[PRIORITY1]
      ,[PURPOSE2]
      ,[LOCATION2]
      ,[TYPE2]
      ,[PRIORITY2]
      ,[PURPOSE3]
      ,[LOCATION3]
      ,[TYPE3]
      ,[PRIORITY3]
      ,[PURPOSE4]
      ,[LOCATION4]
      ,[TYPE4]
      ,[PRIORITY4]
      ,[PURPOSE5]
      ,[LOCATION5]
      ,[TYPE5]
      ,[PRIORITY5]
      ,[PURPOSE6]
      ,[LOCATION6]
      ,[TYPE6]
      ,[PRIORITY6]
      ,[PURPOSE7]
      ,[LOCATION7]
      ,[TYPE7]
      ,[PRIORITY7]
      ,[PURPOSE8]
      ,[LOCATION8]
      ,[TYPE8]
      ,[PRIORITY8]
      ,[PURPOSE9]
      ,[LOCATION9]
      ,[TYPE9]
      ,[PRIORITY9]
      ,[PURPOSE10]
      ,[LOCATION10]
      ,[TYPE10]
      ,[PRIORITY10]
      ,[SUPPLIERNAMEOLD]
      ,[SUPPLIERPARTNUMOLD]
      ,[UOMOLD]
      ,[AttributeData]
      ,[_computed_normalized_suppliername_]
      ,[_katalog_inload_]
      ,[_computed_normalized_art_nr_]
      ,[_artikel_key_inload_]
      ,[_csv_file_]
      ,[_prio_flag_]
      ,[_hibc_]
      ,[_gtin_]
      ,[_ean_kommentar_]
      ,[_artikel_key_]
          ,[_LieferantenNameGlobal_]
          ,[_AGKA_Kreditoren_ID_]
          ,[_artikel_key_class_]
          ,[_agka_class_id_]
          ,[_UStID_Vorschlag_]
          ,[_gtin_pruefziffer_]
          ,[_gtin_pruefung_kommentar_]

FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] tbl_roh
JOIN cte_warenkorb_map 
ON tbl_roh._artikel_key_ = cte_warenkorb_map.ARTIKEL_KEY_WARENKORB
WHERE tbl_roh.[_prio_flag_] NOT LIKE '0'
AND tbl_roh.[_LieferantenNameGlobal_] = cte_warenkorb_map.[LieferantenNameGlobal]
)
SELECT 
    tbl_war.[Index]
    ,[LieferantenNrAGKA]
    ,[LieferantenNameGlobal]
    ,[LieferantenArtikelNr]
    ,[LieferantenArtikelBeschreibung]
    ,ghx_kompl.[NOU]
    ,ghx_kompl.[UOM]
    ,ghx_kompl.ARTIKEL_KEY_WARENKORB
    ,[TPSHORTNAME]
    ,[ACTIONCODE]
    ,[LOCALE]
    ,[MFRNAME]
    ,[MFRPARTNUM]
    ,[SUPPLIERNAME]
    ,[SUPPLIERPARTNUM]
    ,[BaseUOM]
    ,ghx_kompl.[NOU]
    ,ghx_kompl.[UOM]
    ,[SHORTDESC]
    ,[LISTPRICE]
    ,[TAXWARECOMMODITYCODE]
    ,[LONGDESC]
    ,[SEARCHABLEKEYWORD]
    ,[EUROPMEDICALDEVICEDIRECTIVECLASS]
    ,[PACKAGINGINFORMATION]
    ,[EAN]
    ,[PZN]
    ,[LOTSIZE]
    ,[MINORDER]
    ,[MAXORDER]
    ,[LEADTIME]
    ,[AVAILABILITY]
    ,[NETWEIGHT]
    ,[GROSSWEIGHT]
    ,[WEIGHTUOM]
    ,[PRODUCTWIDTH]
    ,[PRODUCTHEIGHT]
    ,[PRODUCTDEPTH]
    ,[PACKAGEDWIDTH]
    ,[PACKAGEDHEIGHT]
    ,[PACKAGEDDEPTH]
    ,[DIMENSIONUOM]
    ,[CATLEVEL1]
    ,[CATLEVEL2]
    ,[CATLEVEL3]
    ,[CATLEVEL4]
    ,[CATLEVEL5]
    ,[CATNAME]
    ,[CATVERSION]
    ,[CATCODE]
    ,[OTHERPARTNUMNAME1]
    ,[OTHERPARTNUM1]
    ,[OTHERPARTNUMNAME2]
    ,[OTHERPARTNUM2]
    ,[DIVISION]
    ,[HAZMATFLAG]
    ,[RECYCLEDMATERIALS]
    ,[SAFETYFLAG]
    ,[RESTRICTFLAG]
    ,[RESTRICTTYPES]
    ,[PURPOSE1]
    ,[LOCATION1]
    ,[TYPE1]
    ,[PRIORITY1]
    ,[PURPOSE2]
    ,[LOCATION2]
    ,[TYPE2]
    ,[PRIORITY2]
    ,[PURPOSE3]
    ,[LOCATION3]
    ,[TYPE3]
    ,[PRIORITY3]
    ,[PURPOSE4]
    ,[LOCATION4]
    ,[TYPE4]
    ,[PRIORITY4]
    ,[PURPOSE5]
    ,[LOCATION5]
    ,[TYPE5]
    ,[PRIORITY5]
    ,[PURPOSE6]
    ,[LOCATION6]
    ,[TYPE6]
    ,[PRIORITY6]
    ,[PURPOSE7]
    ,[LOCATION7]
    ,[TYPE7]
    ,[PRIORITY7]
    ,[PURPOSE8]
    ,[LOCATION8]
    ,[TYPE8]
    ,[PRIORITY8]
    ,[PURPOSE9]
    ,[LOCATION9]
    ,[TYPE9]
    ,[PRIORITY9]
    ,[PURPOSE10]
    ,[LOCATION10]
    ,[TYPE10]
    ,[PRIORITY10]
    ,[SUPPLIERNAMEOLD]
    ,[SUPPLIERPARTNUMOLD]
    ,[UOMOLD]
    ,[AttributeData]
    ,[_computed_normalized_suppliername_]
    ,[_katalog_inload_]
    ,[_computed_normalized_art_nr_]
    ,[_artikel_key_inload_]
    ,[_csv_file_]
    ,[_prio_flag_]
    ,[_hibc_]
    ,[_gtin_]
    ,[_ean_kommentar_]
    ,[_artikel_key_]
      ,[_LieferantenNameGlobal_]
      ,[_artikel_key_]
      ,[_AGKA_Kreditoren_ID_]
      ,[_artikel_key_class_]
      ,[_agka_class_id_]
      ,[_UStID_Vorschlag_]
      ,[_gtin_pruefziffer_]
      ,[_gtin_pruefung_kommentar_]

FROM [Vorlauf_DB].[dbo].[tbl_warenkorb] tbl_war
LEFT JOIN  ghx_kompl
ON   ghx_kompl.[Index] = tbl_war.[Index]
"""

### Warenkorb: Mapping eCl@ss Ebene 

In [92]:
sql_export_warenkorb = """
WITH cte_warenkorb_map AS (
SELECT [Index]
,[LieferantenNrAGKA]
,[LieferantenNameGlobal]
,[LieferantenArtikelNr]
,[LieferantenArtikelBeschreibung]
,[NOU]
,[UOM]
,CONCAT(LieferantenNameGlobal,'°',LieferantenArtikelNr) AS ARTIKEL_KEY_WARENKORB
FROM [Vorlauf_DB].[dbo].[tbl_warenkorb]
),
ghx_kompl AS (
SELECT cte_warenkorb_map.[Index]
      ,cte_warenkorb_map.ARTIKEL_KEY_WARENKORB
      ,[TPSHORTNAME]
      ,[ACTIONCODE]
      ,[LOCALE]
      ,[MFRNAME]
      ,[MFRPARTNUM]
      ,[SUPPLIERNAME]
      ,[SUPPLIERPARTNUM]
      ,[BaseUOM]
      ,tbl_roh.[NOU]
      ,tbl_roh.[UOM]
      ,[SHORTDESC]
      ,[LISTPRICE]
      ,[TAXWARECOMMODITYCODE]
      ,[LONGDESC]
      ,[SEARCHABLEKEYWORD]
      ,[EUROPMEDICALDEVICEDIRECTIVECLASS]
      ,[PACKAGINGINFORMATION]
      ,[EAN]
      ,[PZN]
      ,[LOTSIZE]
      ,[MINORDER]
      ,[MAXORDER]
      ,[LEADTIME]
      ,[AVAILABILITY]
      ,[NETWEIGHT]
      ,[GROSSWEIGHT]
      ,[WEIGHTUOM]
      ,[PRODUCTWIDTH]
      ,[PRODUCTHEIGHT]
      ,[PRODUCTDEPTH]
      ,[PACKAGEDWIDTH]
      ,[PACKAGEDHEIGHT]
      ,[PACKAGEDDEPTH]
      ,[DIMENSIONUOM]
      ,[CATLEVEL1]
      ,[CATLEVEL2]
      ,[CATLEVEL3]
      ,[CATLEVEL4]
      ,[CATLEVEL5]
      ,[CATNAME]
      ,[CATVERSION]
      ,[CATCODE]
      ,[OTHERPARTNUMNAME1]
      ,[OTHERPARTNUM1]
      ,[OTHERPARTNUMNAME2]
      ,[OTHERPARTNUM2]
      ,[DIVISION]
      ,[HAZMATFLAG]
      ,[RECYCLEDMATERIALS]
      ,[SAFETYFLAG]
      ,[RESTRICTFLAG]
      ,[RESTRICTTYPES]
      ,[PURPOSE1]
      ,[LOCATION1]
      ,[TYPE1]
      ,[PRIORITY1]
      ,[PURPOSE2]
      ,[LOCATION2]
      ,[TYPE2]
      ,[PRIORITY2]
      ,[PURPOSE3]
      ,[LOCATION3]
      ,[TYPE3]
      ,[PRIORITY3]
      ,[PURPOSE4]
      ,[LOCATION4]
      ,[TYPE4]
      ,[PRIORITY4]
      ,[PURPOSE5]
      ,[LOCATION5]
      ,[TYPE5]
      ,[PRIORITY5]
      ,[PURPOSE6]
      ,[LOCATION6]
      ,[TYPE6]
      ,[PRIORITY6]
      ,[PURPOSE7]
      ,[LOCATION7]
      ,[TYPE7]
      ,[PRIORITY7]
      ,[PURPOSE8]
      ,[LOCATION8]
      ,[TYPE8]
      ,[PRIORITY8]
      ,[PURPOSE9]
      ,[LOCATION9]
      ,[TYPE9]
      ,[PRIORITY9]
      ,[PURPOSE10]
      ,[LOCATION10]
      ,[TYPE10]
      ,[PRIORITY10]
      ,[SUPPLIERNAMEOLD]
      ,[SUPPLIERPARTNUMOLD]
      ,[UOMOLD]
      ,[AttributeData]
      ,[_computed_normalized_suppliername_]
      ,[_katalog_inload_]
      ,[_computed_normalized_art_nr_]
      ,[_artikel_key_inload_]
      ,[_csv_file_]
      ,[_prio_flag_]
      ,[_hibc_]
      ,[_gtin_]
      ,[_ean_kommentar_]
      ,[_artikel_key_]
      ,[_LieferantenNameGlobal_]
      ,[_AGKA_Kreditoren_ID_]
      ,[_artikel_key_class_]
      ,[_agka_class_id_]
      ,[_UStID_Vorschlag_]
      ,[_gtin_pruefziffer_]
      ,[_gtin_pruefung_kommentar_]
      ,[_AGKclassCode-LEVEL1_]
      ,[_AGKclassCode-LEVEL2_]
FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] tbl_roh
JOIN cte_warenkorb_map 
ON CONCAT(tbl_roh.[_LieferantenNameGlobal_],'°',tbl_roh._computed_normalized_art_nr_) = cte_warenkorb_map.ARTIKEL_KEY_WARENKORB
WHERE tbl_roh.[_prio_flag_] NOT LIKE '0'
AND tbl_roh.[_LieferantenNameGlobal_] = cte_warenkorb_map.[LieferantenNameGlobal]
)
SELECT 
    tbl_war.[Index]
    ,[LieferantenNrAGKA]
    ,[LieferantenNameGlobal]
    ,[LieferantenArtikelNr]
    ,[LieferantenArtikelBeschreibung]
    ,ghx_kompl.[NOU]
    ,ghx_kompl.[UOM]
    ,ghx_kompl.ARTIKEL_KEY_WARENKORB
    ,[TPSHORTNAME]
    ,[ACTIONCODE]
    ,[LOCALE]
    ,[MFRNAME]
    ,[MFRPARTNUM]
    ,[SUPPLIERNAME]
    ,[SUPPLIERPARTNUM]
    ,[BaseUOM]
    ,ghx_kompl.[NOU]
    ,ghx_kompl.[UOM]
    ,[SHORTDESC]
    ,[LISTPRICE]
    ,[TAXWARECOMMODITYCODE]
    ,[LONGDESC]
    ,[SEARCHABLEKEYWORD]
    ,[EUROPMEDICALDEVICEDIRECTIVECLASS]
    ,[PACKAGINGINFORMATION]
    ,[EAN]
    ,[PZN]
    ,[LOTSIZE]
    ,[MINORDER]
    ,[MAXORDER]
    ,[LEADTIME]
    ,[AVAILABILITY]
    ,[NETWEIGHT]
    ,[GROSSWEIGHT]
    ,[WEIGHTUOM]
    ,[PRODUCTWIDTH]
    ,[PRODUCTHEIGHT]
    ,[PRODUCTDEPTH]
    ,[PACKAGEDWIDTH]
    ,[PACKAGEDHEIGHT]
    ,[PACKAGEDDEPTH]
    ,[DIMENSIONUOM]
    ,[CATLEVEL1]
    ,[CATLEVEL2]
    ,[CATLEVEL3]
    ,[CATLEVEL4]
    ,[CATLEVEL5]
    ,[CATNAME]
    ,[CATVERSION]
    ,[CATCODE]
    ,[OTHERPARTNUMNAME1]
    ,[OTHERPARTNUM1]
    ,[OTHERPARTNUMNAME2]
    ,[OTHERPARTNUM2]
    ,[DIVISION]
    ,[HAZMATFLAG]
    ,[RECYCLEDMATERIALS]
    ,[SAFETYFLAG]
    ,[RESTRICTFLAG]
    ,[RESTRICTTYPES]
    ,[PURPOSE1]
    ,[LOCATION1]
    ,[TYPE1]
    ,[PRIORITY1]
    ,[PURPOSE2]
    ,[LOCATION2]
    ,[TYPE2]
    ,[PRIORITY2]
    ,[PURPOSE3]
    ,[LOCATION3]
    ,[TYPE3]
    ,[PRIORITY3]
    ,[PURPOSE4]
    ,[LOCATION4]
    ,[TYPE4]
    ,[PRIORITY4]
    ,[PURPOSE5]
    ,[LOCATION5]
    ,[TYPE5]
    ,[PRIORITY5]
    ,[PURPOSE6]
    ,[LOCATION6]
    ,[TYPE6]
    ,[PRIORITY6]
    ,[PURPOSE7]
    ,[LOCATION7]
    ,[TYPE7]
    ,[PRIORITY7]
    ,[PURPOSE8]
    ,[LOCATION8]
    ,[TYPE8]
    ,[PRIORITY8]
    ,[PURPOSE9]
    ,[LOCATION9]
    ,[TYPE9]
    ,[PRIORITY9]
    ,[PURPOSE10]
    ,[LOCATION10]
    ,[TYPE10]
    ,[PRIORITY10]
    ,[SUPPLIERNAMEOLD]
    ,[SUPPLIERPARTNUMOLD]
    ,[UOMOLD]
    ,[AttributeData]
    ,[_computed_normalized_suppliername_]
    ,[_katalog_inload_]
    ,[_computed_normalized_art_nr_]
    ,[_artikel_key_inload_]
    ,[_csv_file_]
    ,[_prio_flag_]
    ,[_hibc_]
    ,[_gtin_]
    ,[_ean_kommentar_]
    ,[_artikel_key_]
      ,[_LieferantenNameGlobal_]
      ,[_AGKA_Kreditoren_ID_]
      ,[_artikel_key_class_]
      ,[_agka_class_id_]
      ,[_UStID_Vorschlag_]
      ,[_gtin_pruefziffer_]
      ,[_gtin_pruefung_kommentar_]
      ,[_AGKclassCode-LEVEL1_]
      ,[_AGKclassCode-LEVEL2_]
FROM [Vorlauf_DB].[dbo].[tbl_warenkorb] tbl_war
LEFT JOIN  ghx_kompl
ON   ghx_kompl.[Index] = tbl_war.[Index]
"""

In [93]:
sql_set_import_flag = """   with cte AS (
                            SELECT COUNT(*) Anz_Datensaetze
                            ,[Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_artikel_key_inload_]
                            FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                            GROUP BY [_artikel_key_inload_]),
                            cte2 AS (
                            SELECT 
                            [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_inload_  
                            ,_aktuellster_inload_ = MAX(_katalog_inload_)
                            FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                            GROUP BY _artikel_key_inload_),
                            cte_master AS (
                            SELECT  [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_inload_
                            ,[Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._katalog_inload_
                            ,cte.Anz_Datensaetze
                            ,CASE WHEN cte.Anz_Datensaetze > 1 
                            AND [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._katalog_inload_ = cte2._aktuellster_inload_
                            THEN '1' 
                            WHEN cte.Anz_Datensaetze = 1 THEN '1'
                            ELSE '0' END AS _prio_flag_	
                            FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                            Left JOIN cte
                            ON cte._artikel_key_inload_ = [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_inload_
                            left JOIN cte2 
                            ON cte2._artikel_key_inload_ = [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_inload_)
                            
                            UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                            SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._prio_flag_ = cte_master._prio_flag_
                            FROM cte_master
                            LEFT JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                            ON [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_inload_= cte_master._artikel_key_inload_
                            AND [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._katalog_inload_ = cte_master._katalog_inload_
                            """

In [94]:
# Lösche leere Zeilen
sql_delete_empty_rows = """
                        DELETE FROM {}
                        WHERE 
                                  [TPSHORTNAME] IS NULL OR [TPSHORTNAME] = ''
                              AND [ACTIONCODE] IS NULL OR [TPSHORTNAME] = ''
                              AND [MFRNAME] IS NULL OR [TPSHORTNAME] = ''
                              AND [MFRPARTNUM] IS NULL OR [TPSHORTNAME] = ''
                              AND [SUPPLIERNAME] IS NULL OR [TPSHORTNAME] = ''
                              AND [SUPPLIERPARTNUM] IS NULL OR [TPSHORTNAME] = ''
                              AND [BaseUOM] IS NULL OR [TPSHORTNAME] = ''
                              AND [NOU] IS NULL OR [TPSHORTNAME] = ''
                              AND [UOM] IS NULL OR [TPSHORTNAME] = ''
                        """.format(db_tabellen[0])

In [95]:
sql_ean_handling_master = """
UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
set _hibc_ = NULL
    ,_gtin_ = NULL
    ,_gtin_pruefung_kommentar_ = NULL
    ,_ean_kommentar_ = NULL
    ,_gtin_pruefziffer_ = NULL
	
UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
set _gtin_ = LTRIM(RTRIM(EAN))
from [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
where LTRIM(RTRIM(EAN)) IS NOT NULL
and LTRIM(RTRIM(EAN)) not like '%[^0-9]%'
and DATALENGTH(LTRIM(RTRIM(EAN))) <= '14'
and DATALENGTH(LTRIM(RTRIM(EAN))) >= '8'

UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
set _gtin_pruefziffer_ = dbo._GTIN_CHECK_(_gtin_)

UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
set _gtin_pruefung_kommentar_ = 
    case 
    when _gtin_pruefziffer_ = '0' 
    then 'Von GHX / vom Lieferanten angegebene Prüfziffer ist - fehlerhaft' 
    when _gtin_pruefziffer_ = '1' then 'Von GHX / vom Lieferanten angegebene Prüfziffer ist - korrekt'
    else 'Keine gtin vorhanden' end


UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
set _hibc_ = LTRIM(RTRIM(EAN)) 
from [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
where LTRIM(RTRIM(EAN)) IS NOT NULL      
and LTRIM(RTRIM(EAN)) LIKE '+%'


UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
SET [_ean_kommentar_] = CASE
    when LTRIM(RTRIM(EAN)) IS NOT NULL
    and LTRIM(RTRIM(EAN)) not like '%[^0-9]%'
    and DATALENGTH(LTRIM(RTRIM(EAN))) <= '14'
    and DATALENGTH(LTRIM(RTRIM(EAN))) >= '8'
    THEN 'GTIN'
    when LTRIM(RTRIM(EAN)) IS NOT NULL
    and LTRIM(RTRIM(EAN)) LIKE '+%'
    THEN 'HIBC'
    WHEN LTRIM(RTRIM(EAN)) LIKE	'%;%'
    THEN 'Mehrere Inhalte für diese Zeile angegeben'
    WHEN LTRIM(RTRIM(EAN)) IS NULL
    THEN 'Keine Angabe zu EAN'
    ELSE 'Für diese Zeile, bisher keine Prüfung'
    END
"""

In [96]:
# Ersetzt durch EAN_MASTER 
sql_ean_komm = """ UPDATE {}
                SET [_ean_kommentar_] = CASE
                    WHEN LTRIM(RTRIM(EAN)) NOT LIKE '%[a-zA-Z/.;:<=>@\*_+~ ]%'
                    THEN 'GTIN'
                    WHEN LTRIM(RTRIM(EAN)) LIKE '+%'
                    THEN 'HIBC'
                    WHEN LTRIM(RTRIM(EAN)) LIKE	'%;%'
                    THEN 'Mehrere Inhalte für diese Zeile angegeben'
                    WHEN LTRIM(RTRIM(EAN)) IS NULL
                    THEN 'Keine Angabe zu EAN'
                    ELSE 'Für diese Zeile, bisher keine Prüfung'
                    END
                """.format(db_tabellen[0])

In [97]:
# Ersetzt durch EAN_MASTER
sql_split_ean = """
                UPDATE {}
                SET
                    _gtin_ = CASE
                    WHEN LTRIM(RTRIM(EAN)) NOT LIKE '%[a-zA-Z/.;:<=>@\*_+~- ]%'
                    THEN LTRIM(RTRIM(EAN))
                    END
                    ,_hibc_ = CASE
                    WHEN LTRIM(RTRIM(EAN)) LIKE '+%' 
                    THEN LTRIM(RTRIM(EAN))
                    END            
                """.format(db_tabellen[0])

In [98]:
# Ersetzt durch EAN_MASTER
sql_gtin_pruefz = """
                WITH cte_gtin_pruefz AS (
                SELECT _gtin_
                       ,dbo._GTIN_CHECK_(_gtin_) _gtin_pruefziffer_
                FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                WHERE _gtin_ IS NOT NULL 
                       )
                
                UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
                SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._gtin_pruefziffer_ = cte_gtin_pruefz._gtin_pruefziffer_
                FROM cte_gtin_pruefz
                LEFT JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] 
                ON [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._gtin_ =  cte_gtin_pruefz._gtin_                                                                 
                """

### Globaler Artikel-Schlüssel

#### Schritt 1) Globaler Artikel-Schlüssel: Übertrag Lieferant Global und Kreditoren ID Schlüssel TPSHortname

In [99]:
sql_liefID_liefGlobal_kataloge = """
WITH cte_lief AS (
SELECT lief_kat.TPSHORTNAME
,lief.LieferantenNr
,lief.MiiLieferantGlobal
,lief.AGKALieferanten_Lieferant
,lief.AGKAMED_Langname
,lief.Finale_UStID
FROM [tbl_lieferanten] lief
JOIN [tbl_lieferanten_kataloge] lief_kat
ON lief.[LieferantenNr] = lief_kat.[LieferantenNr])
UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] 
SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._LieferantenNameGlobal_ =  cte_lief.MiiLieferantGlobal,
    [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._AGKA_Kreditoren_ID_ = cte_lief.LieferantenNr,
    [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._UStID_Vorschlag_ = cte_lief.Finale_UStID
FROM cte_lief
LEFT JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] 
ON [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].TPSHORTNAME =  cte_lief.TPSHORTNAME
"""

#### Schritt 2) Globaler Artikel-Schlüssel: Erstellung 

In [100]:
sql_artikel_key_kataloge = """
WITH cte_key_field AS ( 
SELECT   _LieferantenNameGlobal_
,[_computed_normalized_art_nr_]
,[UOM]
,[NOU]
,CONCAT(_LieferantenNameGlobal_,'°',
[_computed_normalized_art_nr_],'°',
[UOM],'°',
[NOU]) AS _artikel_key_
FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh])

UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_ = cte_key_field._artikel_key_
FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
JOIN cte_key_field
ON cte_key_field._LieferantenNameGlobal_ = [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._LieferantenNameGlobal_
AND cte_key_field.[_computed_normalized_art_nr_] = [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_computed_normalized_art_nr_]
AND cte_key_field.[UOM] = [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[UOM]
AND cte_key_field.[NOU] = [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[NOU]"""

### AGKAMED Klassifikation JD

#### Schritt 1) Klassifikation JD: Erstellung artikel_key für Übertrag agka_class in tbl_artikel_agkaclass

In [101]:
sql_agka_class_id = """
WITH cte_agka_class_id_ AS (
SELECT [Lieferantennummer_vom_Lieferanten]
,[Artikelnummer_vom_Lieferanten]
,tbl_art_class.[AGKA_Klassifikation_ergaenzt_korrigiert_JD]
FROM [Vorlauf_DB].[dbo].[tbl_artikel_agkaClass_jd] tbl_art_class)
UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._agka_class_id_ = cte_agka_class_id_.[AGKA_Klassifikation_ergaenzt_korrigiert_JD]
FROM cte_agka_class_id_
RIGHT JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
ON  [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_AGKA_Kreditoren_ID_] = cte_agka_class_id_.[Lieferantennummer_vom_Lieferanten]
AND [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_computed_normalized_art_nr_] = cte_agka_class_id_.[Artikelnummer_vom_Lieferanten]
"""

#### Schritt 2)  Klassifikation JD: Erstellung artikel_key für Übertrag agka_class in tbl_kataloge

In [102]:
sql_artikel_key_kataloge_class = """
WITH cte_art_key_class_roh AS ( 
SELECT 
[_computed_normalized_art_nr_]
,[_AGKA_Kreditoren_ID_]
,CONCAT([_AGKA_Kreditoren_ID_], '°',[_computed_normalized_art_nr_]) _artikel_key_class_
FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh])
UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_class_ = cte_art_key_class_roh._artikel_key_class_
FROM cte_art_key_class_roh
JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
ON [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._computed_normalized_art_nr_ = cte_art_key_class_roh._computed_normalized_art_nr_
AND [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_AGKA_Kreditoren_ID_] = cte_art_key_class_roh.[_AGKA_Kreditoren_ID_]
"""

#### Schritt 3)  Klassifikation JD: Übertrag agka_class_id JOIN über Artikel_class_key

In [103]:
sql_transfer_agka_class = """
WITH cte_transfer_agka_class AS ( 
SELECT _artikel_key_class_
,AGKA_Klassifikation_ergaenzt_korrigiert_JD
FROM tbl_artikel_agkaClass_jd)

UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._agka_class_id_ = cte_transfer_agka_class.AGKA_Klassifikation_ergaenzt_korrigiert_JD
FROM cte_transfer_agka_class
JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
ON [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]._artikel_key_class_ = cte_transfer_agka_class._artikel_key_class_
"""

In [104]:
sql_agkaClass_Bez_in_ghx = """

WITH cte_agkaClass_baum AS (

SELECT [AgkClassID]
      ,[AGKclassCode-LEVEL1]
      ,[AGKclassCode-LEVEL2]
      ,[AGKclassCode-LEVEL3]
      ,[AGKclassCode-LEVEL4]
 FROM [Vorlauf_DB].[dbo].[tbl_agkaClass_baum]),

 cte_ghx_roh AS (
 SELECT [_agka_class_id_]
  FROM [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]),
  
cte_roh_inkl_agka AS (
SELECT  cte_ghx_roh.[_agka_class_id_] 
,cte_agkaClass_baum.[AGKclassCode-LEVEL1]
,cte_agkaClass_baum.[AGKclassCode-LEVEL2]
,cte_agkaClass_baum.[AGKclassCode-LEVEL3]
,cte_agkaClass_baum.[AGKclassCode-LEVEL4]
FROM cte_ghx_roh
JOIN cte_agkaClass_baum
ON cte_ghx_roh.[_agka_class_id_] = cte_agkaClass_baum.[AgkClassID])

UPDATE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] 
SET [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_AGKclassCode-LEVEL1_] = cte_roh_inkl_agka.[AGKclassCode-LEVEL1]
    ,[Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_AGKclassCode-LEVEL2_] = cte_roh_inkl_agka.[AGKclassCode-LEVEL2]
    ,[Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_AGKclassCode-LEVEL3_] = cte_roh_inkl_agka.[AGKclassCode-LEVEL3]
    ,[Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_AGKclassCode-LEVEL4_] = cte_roh_inkl_agka.[AGKclassCode-LEVEL4]
FROM cte_roh_inkl_agka
LEFT JOIN [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh] 
ON [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh].[_agka_class_id_] =  cte_roh_inkl_agka.[_agka_class_id_]     
   
"""   

In [105]:
# # Index erstellen (Autoincrement je Tabelle)
sql_handle_kataloge_id = """
IF EXISTS (SELECT TABLE_NAME
               FROM   Vorlauf_DB.INFORMATION_SCHEMA.COLUMNS
               WHERE  TABLE_NAME = 'tbl_ghx_kataloge_roh'
                      AND COLUMN_NAME = 'tbl_index'
                      AND TABLE_SCHEMA='DBO')
  BEGIN
      ALTER TABLE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
      DROP COLUMN tbl_index
      PRINT 'Spalte tbl_index gelöscht.'
      ALTER TABLE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
      ADD tbl_index INT NOT NULL IDENTITY (1,1)
      PRINT 'Spalte nach dem Löschen hinzugefügt'
  END
ELSE 
BEGIN
    ALTER TABLE [Vorlauf_DB].[dbo].[tbl_ghx_kataloge_roh]
    ADD tbl_index INT NOT NULL IDENTITY (1,1)
    PRINT 'Spalte tbl_index inkl. auto_increment Werte hinzugefügt.'
END
"""

### Funktionen zur Steuerung SQL-Querys

In [106]:
def _sql_globaler_artikel():
    """ Update GHX-Kataloge: Erstelle Globalen Artikel """
    start = time.process_time()
    _sql_executer(sql_liefID_liefGlobal_kataloge)
    _sql_executer(sql_artikel_key_kataloge)
    print("\nGlobaler Artikel-Key wurde erstellt.")
    print(time.process_time() - start)  


def _sql_agka_class():
    """ Update GHX-Kataloge / artikel-agkaClass-JD: AGKAMED-Class """
    start = time.process_time()
    _sql_executer(sql_agka_class_id)
    _sql_executer(sql_artikel_key_kataloge_class)
    _sql_executer(sql_transfer_agka_class)
    _sql_executer(sql_agkaClass_Bez_in_ghx)

    print("\nAGKAMED Class-ID in GHX-Kataloge wurde übertragen.")
    print(time.process_time() - start)  


def _sql_import_kataloge():
    """ Obligatorisch bei Katalog-Import"""
    start = time.process_time()
    _sql_executer(sql_delete_empty_rows)
    _sql_executer(sql_set_import_flag)
    _sql_executer(sql_ean_handling_master)
    # ##########################
    # Wird ersetzt durch master
    #_sql_executer(sql_ean_komm)
    #_sql_executer(sql_split_ean) 
    #_sql_executer(sql_gtin_pruefz) # GTIN-Prüfziffer Testlauf
    # ##########################
    _sql_executer(sql_handle_kataloge_id)
    print("\nObligatorische Anpassung nach ZIP Import wurde durchgeführt.")
    print(time.process_time() - start)  


## Steuerungsfunktionen

In [107]:
def zip_steuerung():
    """
    Steuerung der Ausführung gewünschter Funktionen
    
    Funktion zum Ausführen der Schritte kopieren und entpacken der zips, \n
    einladen in DB, löschen der Ordner in einem Schritt. 
    Erforderlich um Scope auf relevante zip-Dateien in beiden Prozessen gleich zu halten.
    
    INPUT:
        --
    OUTPUT:
        --
    """
    start_zip_handling = _start_zip_handler()
    
    if start_zip_handling == False:
        try:
            print("Ordnerinhalte ungleich. Zip Handler wird gestartet.")
            df_relev_zips = pd.DataFrame(_erstelle_inload_liste(),dtype=str) 
            _kopiere_relev_zips_from_smb() 
            _extract_folder_from_zips(df_relev_zips,'Zip_Name_ID')
        except Exception as e:
            print("Achtung: Scheinbar fehlt im Arbeitsordner nicht die letzte .zip-Datei, sondern eine zwischendrin."
                  "\nDas führt zum Absturz des Programms, da der Importer start jedoch nicht die korrekte zip identifiziert")
        
    else: 
        print("Ordnerinhalte gleich. Durchlauf wird beendet, da Arbeitsordner aktuell ist"
              "\nund keine Updates vorhanden sind.")

In [108]:
def sql_ausfuehrung():
    """
    Steuerung / Ausführung SQL-Skripte: Tabellen-Anpassungen
    INPUT:
        --
    OUTPUT:
        --
    """
    try:
        _sql_import_kataloge() 
        _sql_globaler_artikel()
#        _sql_agka_class() 
    except Exception as e:
        print('Fehler bei der Ausführung der SQL-Skripte aufgetreten')
        print(e)
        
        
    ## METATABELLE COVIN 
#     _sql_executer(sql_cr_tbl_verw_inload_kataloge)
#     _sql_executer(sql_bef_tbl_verw_inload_kataloge)




In [109]:
def stammdaten_tabs_steuerung():
    """
    Steuerung / Ausführung SQL-Skripte: Import xlsx.
    Excel-Tabellen werden regelmäßig geupdated und in DB ersetzt.
    INPUT:
        --
    OUTPUT:
        --
    """
    
    ## ##############
    ## IMPORT-PROZESS
    ## ##############
    
    try:
        #_fill_xlsx_in_ms_sql('tbl_covin_validierung') # Bei Komplett Inload
        #_fill_xlsx_in_ms_sql('tbl_lieferanten_master') # Bei Komplett Inload
        #_fill_xlsx_in_ms_sql('tbl_artikel_agkaClass_jd') # Bei Komplett Inload
        #_fill_xlsx_in_ms_sql('tbl_agkaClass_jd') # Bei Komplett Inload
        #_fill_xlsx_in_ms_sql('tbl_agkaClass_baum') # Bei Komplett Inload

        # _export_warenkorb()
    
    except Exception as e:
        print("Fehler bei Import xlsx.")
        print(e)
        





In [110]:
def main():
    """
    Steuerung aller Funktionen
    """
##     -------------------------------------   
##     1) Verarbeitung / Inload GHX Kataloge
##     -------------------------------------    
#    zip_steuerung() # Für GHX INLOAD aktivieren

##     ----------------------------------------------     
##     2) Verarbeitung / Inload weitere Excel-Dateien
##     ----------------------------------------------      
    stammdaten_tabs_steuerung()

##     --------------------------------------
##     3) Verbindung / Update der DB-Tabellen
##     --------------------------------------
#    sql_ausfuehrung() # Für GHX INLOAD aktivieren

##     -------------------------------------------     
##     Müsste hierher ausgelagert werden, da stammdaten_tabs und sql_querys vorher bearbeitet werden sollten. 
##     Warenkorb-Mapping ist ein separater Prozess
##     -------------------------------------------  
#    _export_warenkorb() 
    
    
    # IMPORT SCHEDULE 

In [111]:

# if __name__ == "__main__":
    # main()
    
   # schedule.every().day.at('03:30').do(main)

    #while True:
    #    schedule.run_pending()
    #    time.sleep(1)

LieferantenNrAGKA
LieferantenNameGlobal
LieferantenArtikelNr

Warenkorb Mapping abgeschlossen.
die KATALOGMAPPING_eCl@ss.xlsx Datei liegt bereit.
292.75


In [1]:
def handling_export_warenkorb():
    _export_warenkorb()

In [ ]:
def handling_export_warenkorb_gtin():
    _export_warenkorb_gtin()